In [1]:
import tensorflow as tf
from matplotlib import pyplot as plt
import matplotlib.image as mpimg
import pandas as pd
import numpy as np

In [2]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [4]:
np.set_printoptions(precision=3)

In [5]:
def display(alist, show=True):
    print('type:%s\nshape: %s' %(alist[0].dtype, alist[0].shape))
    if show:
        for i in range(3):
            print('样本%s\n%s' % (i, alist[i]))

In [6]:
scalars = np.array([1, 2, 3], dtype=int64)

In [17]:
scalars[0]

1

In [8]:
display(scalars)

type:int64
shape: ()
样本0
1
样本1
2
样本2
3


In [11]:
vectors = np.array([[0.1, 0.1, 0.1], [0.2, 0.2, 0.2], [0.3, 0.3, 0.3]], dtype=float32)

In [18]:
vectors[0]

array([0.1, 0.1, 0.1], dtype=float32)

In [13]:
matrices = np.array([np.array((vectors[0],vectors[0])),
                    np.array((vectors[1],vectors[1])),
                    np.array((vectors[2],vectors[2]))],dtype=float32)

In [14]:
matrices

array([[[0.1, 0.1, 0.1],
        [0.1, 0.1, 0.1]],

       [[0.2, 0.2, 0.2],
        [0.2, 0.2, 0.2]],

       [[0.3, 0.3, 0.3],
        [0.3, 0.3, 0.3]]], dtype=float32)

In [15]:
display(matrices)

type:float32
shape: (2, 3)
样本0
[[0.1 0.1 0.1]
 [0.1 0.1 0.1]]
样本1
[[0.2 0.2 0.2]
 [0.2 0.2 0.2]]
样本2
[[0.3 0.3 0.3]
 [0.3 0.3 0.3]]


In [16]:
writer = tf.python_io.TFRecordWriter('%s.tfrecord' % 'test')

In [21]:
for i in range(3):
    features = {}
    features['scalar'] = tf.train.Feature(int64_list=tf.train.Int64List(value=[scalars[i]]))
    
    features['vector'] = tf.train.Feature(float_list=tf.train.FloatList(value=vectors[i]))
    
    features['matrix'] = tf.train.Feature(float_list=tf.train.FloatList(value=matrices[i].reshape(-1)))
    features['matrix_shape'] = tf.train.Feature(int64_list=tf.train.Int64List(value=matrices[i].shape))

In [22]:
features

{'scalar': int64_list {
   value: 3
 }, 'vector': float_list {
   value: 0.30000001192092896
   value: 0.30000001192092896
   value: 0.30000001192092896
 }, 'matrix': float_list {
   value: 0.30000001192092896
   value: 0.30000001192092896
   value: 0.30000001192092896
   value: 0.30000001192092896
   value: 0.30000001192092896
   value: 0.30000001192092896
 }, 'matrix_shape': int64_list {
   value: 2
   value: 3
 }}

In [23]:
tf_features = tf.train.Features(feature=features)

In [25]:
tf_example = tf.train.Example(features=tf_features)

In [28]:
tf_serialized = tf_example.SerializeToString()

In [29]:
writer.write(tf_serialized)

In [30]:
writer.close()

In [31]:
filenames = ["test.tfrecord", "test.tfrecord"]

In [32]:
dataset = tf.data.TFRecordDataset(filenames)

In [2]:
from tfrecorder import TFrecorder

In [3]:
mnist = tf.contrib.learn.datasets.load_dataset("mnist")

Instructions for updating:
Please use tf.data.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST-data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST-data/train-labels-idx1-ubyte.gz
Extracting MNIST-data/t10k-images-idx3-ubyte.gz
Extracting MNIST-data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [4]:
mnist

Datasets(train=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x11b5feeb8>, validation=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x11bbf2518>, test=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x11bbf24e0>)

In [5]:
df = pd.DataFrame({'name':['image', 'label'],
                  'type':['float32', 'int64'],
                  'shape':[(784,),()],
                  'isbyte':[False, False],
                  'length_type':['fixed', 'fixed'],
                  'default':[np.NaN, np.NaN]})

In [6]:
tfr = TFrecorder()

In [7]:
dataset = mnist.train

In [8]:
dataset

In [9]:
path = 'mnist_tfrecord/train/train'

In [10]:
num_examples_per_file = 1000

In [11]:
num_so_far = 0

In [12]:
writer = tf.python_io.TFRecordWriter('%s%s_%s.tfrecord' % (path, num_so_far, num_examples_per_file))

In [13]:
for i in np.arange(dataset.num_examples):
    features = {}
    tfr.feature_writer(df.iloc[0], dataset.images[i], features)
    tfr.feature_writer(df.iloc[1], dataset.labels[i], features)
    
    tf_features = tf.train.Features(feature=features)
    tf_example = tf.train.Example(features=tf_features)
    tf_serialized = tf_example.SerializeToString()
    writer.write(tf_serialized)
    
    if i%num_examples_per_file == 0 and i!=0:
        writer.close()
        num_so_far = i
        writer = tf.python_io.TFRecordWriter('%s%s_%s.tfrecord' %(path, num_so_far, i+num_examples_per_file))
        print('saved %s%s_%s.tfrecord' % (path, num_so_far, i+num_examples_per_file))

saved mnist_tfrecord/train/train1000_2000.tfrecord
saved mnist_tfrecord/train/train2000_3000.tfrecord
saved mnist_tfrecord/train/train3000_4000.tfrecord
saved mnist_tfrecord/train/train4000_5000.tfrecord
saved mnist_tfrecord/train/train5000_6000.tfrecord
saved mnist_tfrecord/train/train6000_7000.tfrecord
saved mnist_tfrecord/train/train7000_8000.tfrecord
saved mnist_tfrecord/train/train8000_9000.tfrecord
saved mnist_tfrecord/train/train9000_10000.tfrecord
saved mnist_tfrecord/train/train10000_11000.tfrecord
saved mnist_tfrecord/train/train11000_12000.tfrecord
saved mnist_tfrecord/train/train12000_13000.tfrecord
saved mnist_tfrecord/train/train13000_14000.tfrecord
saved mnist_tfrecord/train/train14000_15000.tfrecord
saved mnist_tfrecord/train/train15000_16000.tfrecord
saved mnist_tfrecord/train/train16000_17000.tfrecord
saved mnist_tfrecord/train/train17000_18000.tfrecord
saved mnist_tfrecord/train/train18000_19000.tfrecord
saved mnist_tfrecord/train/train19000_20000.tfrecord
saved mnis

In [51]:
writer.close()

In [52]:
data_info_path = 'mnist_tfrecord/data_info.csv'
df.to_csv(data_info_path, index=False)

In [53]:
# 用该方法写测试集的tfrecord文件
dataset = mnist.test
path = 'mnist_tfrecord/test/test'
# 每个tfrecord文件写多少个样本
num_examples_per_file = 1000
# 当前写的样本数
num_so_far = 0
# 要写入的文件
writer = tf.python_io.TFRecordWriter('%s%s_%s.tfrecord' %(path, num_so_far, num_examples_per_file))
# 写多个样本
for i in np.arange(dataset.num_examples):
    # 要写到tfrecord文件中的字典
    features = {}
    # 写一个样本的图片信息存到字典features中
    tfr.feature_writer(df.iloc[0], dataset.images[i], features)
    # 写一个样本的标签信息存到字典features中
    tfr.feature_writer(df.iloc[1], dataset.labels[i], features)
    
    tf_features = tf.train.Features(feature= features)
    tf_example = tf.train.Example(features = tf_features)
    tf_serialized = tf_example.SerializeToString()
    writer.write(tf_serialized)
    # 每写了num_examples_per_file个样本就令生成一个tfrecord文件
    if i%num_examples_per_file ==0 and i!=0:
        writer.close()
        num_so_far = i
        writer = tf.python_io.TFRecordWriter('%s%s_%s.tfrecord' %(path, num_so_far, i+num_examples_per_file))
        print('saved %s%s_%s.tfrecord' %(path, num_so_far, i+num_examples_per_file))
writer.close()

saved mnist_tfrecord/test/test1000_2000.tfrecord
saved mnist_tfrecord/test/test2000_3000.tfrecord
saved mnist_tfrecord/test/test3000_4000.tfrecord
saved mnist_tfrecord/test/test4000_5000.tfrecord
saved mnist_tfrecord/test/test5000_6000.tfrecord
saved mnist_tfrecord/test/test6000_7000.tfrecord
saved mnist_tfrecord/test/test7000_8000.tfrecord
saved mnist_tfrecord/test/test8000_9000.tfrecord
saved mnist_tfrecord/test/test9000_10000.tfrecord


In [16]:
def input_fn_maker(path, data_info_path, shuffle=False, batch_size=1, epoch=1, padding=None):
    def input_fn():
        filenames = tfr.get_filenames(path, shuffle=shuffle)
        dataset = tfr.get_dataset(paths=filenames, data_info=data_info_path, shuffle=shuffle,
                                 batch_size=batch_size, epoch=epoch, padding=padding)
        return dataset.make_one_shot_iterator().get_next()
    return input_fn

In [19]:
padding_info = ({'image': [784,], 'label':[]})

In [20]:
test_input_fn = input_fn_maker('mnist_tfrecord/test/', 'mnist_tfrecord/data_info.csv', padding=padding_info)
train_input_fn = input_fn_maker('mnist_tfrecord/train/', 'mnist_tfrecord/data_info.csv', shuffle=True, batch_size=512, padding=padding_info)
train_eval_fn = input_fn_maker('mnist_tfrecord/train/', 'mnist_tfrecord/data_info.csv', batch_size=512, padding=padding_info)

In [ ]:
def model_fn(features, mode):
    features['image'] = tf.reshape(features['image'], [-1, 28,28,1])
    conv1 = tf.layers.conv2d(
        inputs=features['image'],
        filters=32,
        kernel_size=[5, 5],
        padding='same',
        activation=tf.nn.relu,
        name='conv1')
    pool1 = tf.layers.max_pooling2d(inputs=conv1, 
                                                pool_size=[2, 2], 
                                                strides=2,
                                                name='pool1')
    conv2 = tf.layers.conv2d(
        inputs=pool1,
        fliters=64,
        kernel_size=[5, 5],
        padding='same',
        activation=tf.nn.relu,
        name='conv2')
    pool2 = tf.layers.max_pooling2d(inputs=conv2,
                                                pool_size=[2, 2],
                                                strides=2,
                                                name='pool2')
    pool2_flat = tf.reshape(pool2,
                                [-1, 7*7*64],
                               name='pool2_flat')
    dense1 = tf.layers.dense(inputs=pool2_flat,
                                    units=1024,
                                    activation=tf.nn.relu,
                                    name='dense1')
    dropout = tf.layers.dropout(inputs=dense1,
                                           rate=0.4,
                                           training_mode == tf.estimator.ModeKeys.TRAIN)
    logits = tf.layers.dense(input=dropout, units=10, name='output')
    
    predictions = {
        "image": feature['image'],
        "conv1_out":conv1,
        "pool1_out":pool1,
        "conv2_out":conv2,
        "pool2_out":pool2,
        "pool2_flat_out":pool2_flat,
        "dense1_out":dense1,
        "logits":logits,
        "classes":tf.argmax(input=logits, axis=1),
        "labels": features['label'],
        "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)
    
    loss = tf.losses.sparse_softmax_cross_entropy(labels=features['label'], logits=logits)
    if mode = tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.AdamOptimizer(learning_rate=1e-3)
        train_op = optimizer.minimize(loss=loss,
                                        global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)